In [ ]:
# ============================
# BigDataViewer to OpenVDB converter, v1.1
# ----------------------------
# Make sure to open Blender's system console to see conversion progress!

# PATH TO YOUR HDF5 XML FILE
xml_path = "C:/path/to/your/HDF5.xml"

# If you want to differentiate between exports, you can append unique prefixes here
custom_prefix = ""

# This factor is multiplied by the median value found in the timepoint, and the result used as sparsification threshold.
# With many datasets, the median value will be equivalent to the background values.
# By multiplying that median by factor 2, you will include most of the background while not really touching the actual data  
sparse_threshold = 2


# Create a list of timepoints here if you only want to convert single timepoints.
# If this list is empty, all timepoints will be converted.
selected_timepoints = []

# Whether to automatically load the exported VDB timeseries into the scene
append_to_scene = True

# If the h5py package installation fails, reopen Blender with administrator rights.

# ============================


import os
import xml.etree.ElementTree as ET
import openvdb as vdb
import numpy as np
import subprocess
import sys
from pathlib import Path
import bpy

try:
    import h5py
except ModuleNotFoundError:
    python_exe = os.path.join(sys.prefix, 'bin', 'python.exe')
    subprocess.call([python_exe, '-m', 'pip', 'install', '--upgrade', 'h5py'])
    import h5py

print("========== Starting Conversion ============")


# Check if XML file exists
if not os.path.exists(xml_path):
    print(f"Error: XML file {xml_path} not found!")
    raise FileNotFoundError("Could not find XML file!")

tree = ET.parse(xml_path)
root = tree.getroot()

# Get base path
base_path_elem = root.find('.//BasePath')
base_path = base_path_elem.text if base_path_elem is not None else "."

# Get HDF5 file path
h5_elem = root.find('.//ImageLoader/hdf5')
if h5_elem is None:
    raise ValueError("No HDF5 file reference found in XML")

h5_relative = h5_elem.text
xml_dir = os.path.dirname(xml_path)
h5_path = os.path.normpath(os.path.join(xml_dir, base_path, h5_relative))

# Get first ViewSetup (assuming single channel for now)
setup = root.find('.//ViewSetup')
if setup is None:
    raise ValueError("No ViewSetup found in XML")

# Get size
size_elem = setup.find('.//size')
size = [int(x) for x in size_elem.text.split()] if size_elem is not None else None

# Get voxel size
voxel_size_elem = setup.find('.//voxelSize/size')
voxel_size = [float(x) for x in voxel_size_elem.text.split()] if voxel_size_elem is not None else None

# Get unit
unit_elem = setup.find('.//voxelSize/unit')
unit = unit_elem.text if unit_elem is not None else "unknown"

# Get setup ID
setup_id = int(setup.find('.//id').text)

# Get transform
transform_elem = root.find(f'.//ViewRegistration[@setup="{setup_id}"]//affine')
transform = None
if transform_elem is not None:
    transform = [float(x) for x in transform_elem.text.split()]

metadata = {
    'h5_path': h5_path,
    'size': size,
    'voxel_size': voxel_size,
    'unit': unit,
    'setup_id': setup_id,
    'transform': transform
}

print("metadata:")
print(metadata)

# Check if HDF5 file exists
if not os.path.exists(metadata['h5_path']):
    print(f"Error: HDF5 file {metadata['h5_path']} not found!")
    raise FileNotFoundError("No HDF5 file at this location!")


def get_median_and_max_of_timepoint(file: h5py.File, timepoint_string: str, setup_string: str):
    """Return the mean and maximum values of the current timepoint as a tuple"""
    dataset_path = f"{timepoint_string}/{setup_string}/0/cells"
    if dataset_path not in file:
        print(f"Error: Dataset {dataset_path} not found!")
        print(f"Available datasets: {list(file.keys())}")
        raise FileNotFoundError

    data = file[dataset_path][:]
    return (np.mean(data), np.max(data))


def convert_and_save_timepoint(file: h5py.File, timepoint_string: str, setup_string: str, median: float, max: float, thresh: float):
    """Takes a single timepoint, encoded as strings, and exports it to the base directory of the HDF5 XML file."""
    # Read HDF5 file
    # BDV format uses path: t{timepoint}/s{setup}/0/cells
    dataset_path = f"{timepoint_string}/{setup_string}/0/cells"

    print(f"Reading dataset: {dataset_path}")

    if dataset_path not in file:
        print(f"Error: Dataset {dataset_path} not found!")
        print(f"Available datasets: {list(file.keys())}")
        raise FileNotFoundError

    data = file[dataset_path][:]

    # Create OpenVDB grid
    print(f"Creating OpenVDB grid for {dataset_path}...")
    grid = vdb.FloatGrid()
    grid.name = "density"
    grid.background = 0.0

    # Set voxel size (transform) if available
    if metadata['voxel_size']:
        voxel_size = metadata['voxel_size']
        # Create transform matrix for proper scaling
        transform = vdb.Transform()
        
        transform.preScale((voxel_size[2], voxel_size[1], voxel_size[0]))
        
        grid.transform = transform
        print(f"Applied voxel spacing: {voxel_size}")

    data_norm = data.astype(np.float32)
    data_norm = data_norm/max

    # Fill the grid
    tol = thresh * median / max
    grid.clear()
    grid.copyFromArray(data_norm, tolerance=tol)
    grid.saveFloatAsHalf = True
    grid.prune()

    # Generate output path (same directory as XML)
    output_path = os.path.splitext(xml_path)[0] + "_" + custom_prefix + "_" + setup_string + "_" + timepoint_string + ".vdb"

    # Write VDB file
    vdb.write(output_path, grids=[grid])
    print(f"Wrote VDB to output file: {output_path}")
    
    return output_path


f = h5py.File(metadata['h5_path'], 'r')

# Collect all timepoints and setup IDs present in the H5 file
timepoints = [entry for entry in list(f.keys()) if str(entry).startswith("t")]
setup_strings = [entry for entry in list(f.keys()) if str(entry).startswith("s")]

# If the user specified a list of timepoints, only continue with the selection
if len(selected_timepoints) != 0:
    timepoints = [timepoints[i] for i in selected_timepoints]

exported_vdbs = []

# Then, we can read the datasets one by one, normalize and threshold them and export them to VDB
for setup_string in setup_strings:
    # First, we calculate a global median and maximum so we can normalize the dataset globally
    global_median = 0
    global_max = 0
    print(f"Calculating global median and maximum for setup {setup_string}...")
    
    for i, tp in enumerate(timepoints):
        (median, maximum) = get_median_and_max_of_timepoint(f, tp, setup_string)
        global_median += median
        global_max = max(maximum, global_max)
    
        if i % 10 == 0:
            print(f"Finished calculating median/max for {i}/{len(timepoints)} timepoints in {setup_string}")
    
    global_median /= len(timepoints)

    print(f"Global median: {global_median}, global max: {global_max}")
    
    for tp in timepoints:
        path = convert_and_save_timepoint(f, tp, setup_string, global_median, global_max, sparse_threshold)
        exported_vdbs.append(path)
        
print(f"Successfully saved {len(setup_strings)} x {len(timepoints)} VDB files to {xml_dir} \n")

if append_to_scene:
    bpy.ops.object.volume_import(directory=xml_dir, files=[{"name": os.path.basename(path)} for path in exported_vdbs], align='WORLD', location=(0, 0, 0))

metadata:
{'h5_path': 'C:\\CASUS\\datasets\\Pdu_H2BeGFP_CAAXmCherry\\Pdu_H2BeGFP_CAAXmCherry.h5', 'size': [700, 660, 113], 'voxel_size': [1.0, 1.0, 5.0], 'unit': 'pixel', 'setup_id': 0, 'transform': [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0]}
Calculating global median and maximum...
Finished calculating 0/101
Finished calculating 10/101
Finished calculating 20/101
Finished calculating 30/101
Finished calculating 40/101
Finished calculating 50/101
Finished calculating 60/101
Finished calculating 70/101
Finished calculating 80/101
Finished calculating 90/101
Finished calculating 100/101
Global median: 7.656851835815696, global max: 4400
Reading dataset: t00000/s00/0/cells
Creating OpenVDB grid for t00000/s00/0/cells...
Applied voxel spacing: [1.0, 1.0, 5.0]
Wrote VDB to output file: C:\CASUS\datasets\Pdu_H2BeGFP_CAAXmCherry\Pdu_H2BeGFP_CAAXmCherry_thresh3_s00_t00000.vdb
Reading dataset: t00001/s00/0/cells
Creating OpenVDB grid for t00001/s00/0/cells...
Applied voxel spa

: 